In [2]:
import ta 
import sqlalchemy
import pandas as pd 

### Data retrieved from SQLite table engine 

In [3]:
engine = sqlalchemy.create_engine('sqlite:///USA')

In [4]:
names = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',engine)

In [6]:
names[:5]

,name
0,MMM
1,AXP
2,AMGN
3,AAPL
4,BA


In [7]:
names = names.name.to_list()

In [9]:
framelist = []

for name in names:
    framelist.append(pd.read_sql(f'SELECT Date, Close FROM "{name}"',engine))

In [11]:
framelist[0]

,Date,Close
0,1970-01-02 00:00:00.000000,6.851563
1,1970-01-05 00:00:00.000000,6.890625
2,1970-01-06 00:00:00.000000,6.960938
3,1970-01-07 00:00:00.000000,7.000000
4,1970-01-08 00:00:00.000000,7.093750
...,...,...
13170,2022-03-18 00:00:00.000000,148.320007
13171,2022-03-21 00:00:00.000000,148.580002
13172,2022-03-22 00:00:00.000000,149.940002
13173,2022-03-23 00:00:00.000000,147.679993


In [12]:
#function for moving average convergence and divergence 
def MACDdecision(df):
    df['MACD_diff'] = ta.trend.macd_diff(df.Close)
    #to be validated, MACD must be above 0 for the current day and below 0 for the day before
    df.loc[(df.MACD_diff > 0) & (df.MACD_diff.shift(1) < 0), 'Decision_MACD'] = 'Buy'

In [13]:
#function for relative strength index and simple moving average
def RSI_SMAdecision(df):
    df['RSI'] = ta.momentum.rsi(df.Close, window=10)
    df['SMA200'] = ta.trend.sma_indicator(df.Close, window=200)
    #note! RSI below 30 tends to represent an asset that is undervalued 
    df.loc[(df.Close > df.SMA200) & (df.RSI < 30), 'Decision_RSI/SMA'] = 'Buy'

In [14]:
for frame in framelist:
    MACDdecision(frame)
    RSI_SMAdecision(frame)

In [15]:
framelist[0]

,Date,Close,MACD_diff,Decision_MACD,RSI,SMA200,Decision_RSI/SMA
0,1970-01-02 00:00:00.000000,6.851563,NaN,NaN,NaN,NaN,NaN
1,1970-01-05 00:00:00.000000,6.890625,NaN,NaN,NaN,NaN,NaN
2,1970-01-06 00:00:00.000000,6.960938,NaN,NaN,NaN,NaN,NaN
3,1970-01-07 00:00:00.000000,7.000000,NaN,NaN,NaN,NaN,NaN
4,1970-01-08 00:00:00.000000,7.093750,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
13170,2022-03-18 00:00:00.000000,148.320007,1.169610,NaN,51.591604,180.43615,NaN
13171,2022-03-21 00:00:00.000000,148.580002,1.295130,NaN,52.289472,180.14880,NaN
13172,2022-03-22 00:00:00.000000,149.940002,1.432831,NaN,55.978013,179.87985,NaN
13173,2022-03-23 00:00:00.000000,147.679993,1.336664,NaN,48.985453,179.60030,NaN


In [16]:
#check the last row of every df in the list 

for name, frame in zip(names,framelist):
    if frame['Decision_MACD'].iloc[-1] == 'Buy':
        print("Buying signal MACD for " + name)
    if frame['Decision_RSI/SMA'].iloc[-1] == 'Buy':
        print("Buying signal RSI/SMA for " + name)